In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.optimize import minimize

import math

from typing import Tuple

from utils import methods, math_expressions as me

In [2]:
train_df = pd.read_csv('../../data/gen_train_v3.csv')
test_df = pd.read_csv('../../data/gen_test_v3.csv')

In [3]:
train_df.head()

h = 1 / 15
c = 25

In [4]:
train_cols = ['N', 'n', 'mean_n', 'std_n', 'alpha_hat', 'beta_hat', 'u_star_hat']
# train_cols = ['N', 'mean_n', 'u_star_hat', 'beta_hat']

X_train = train_df[train_cols]
y_train = train_df['u']

X_test = test_df[train_cols]
y_test = test_df['u']

if False: 
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = pd.DataFrame(sc.fit_transform(X_train), columns = X_train.columns, index = X_train.index)
    X_test = pd.DataFrame(sc.transform(X_test), columns = X_test.columns, index = X_test.index)


X_train = X_train.values
X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
y_train = y_train.values

X_test = X_test[train_cols].values
X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))
y_test = y_test.values

display(X_train[0])


array([  1.        ,  17.        ,   5.        ,   8.23325294,
         2.06939742,  15.82905947,   0.52013532, 120.01412095])

In [5]:
from sympy import diff, symbols, lambdify
d = symbols('d')
expr = me.cus_cost_expr_1(h, c, d)
expr_grad = diff(expr, d)
expr_hess = diff(expr_grad, d)

display(expr)
display(expr_grad)
display(expr_hess)


if True:
    f = lambdify(d, expr)
    f_grad = lambdify(d, expr_grad)
    f_hess = lambdify(d, expr_hess)

else:
    f = lambda x: float(expr.subs(d, x).evalf(5))
    f_grad = lambda x: float(expr_grad.subs(d, x).evalf(5))
    f_hess = lambda x: float(expr_hess.subs(d, x).evalf(5))

if True:
    f = lambda x: 0.0666*(-x)**1.25 if x <= 0 else 0.0666 * x ** 4
    f_grad = lambda x: 0.08325 * (-x)**0.25 if x <= 0 else 0.2664 * x ** 3

if True:
    f = lambda x: (-x) ** 1 if x <= 0 else (c) * x ** 1
    f_grad = lambda x: h * 1.25 * (-x)**0.25 if x <= 0 else (c / 10) * 1.5 * x ** 0.5

if True:
    f = lambda x: (-x) * h if x <= 0.286980112508514 else c / (1 + math.e ** (-25 * x))
    f_grad = lambda x: h * 1.25 * (-x)**0.25 if x <= 0 else (c / 10) * 1.5 * x ** 0.5

Piecewise((-0.0666666666666667*d, d <= -0.286980112508514), (25/(1 + exp(-25*d)), True))

Piecewise((-0.0666666666666667, d <= -0.286980112508514), (625*exp(-25*d)/(1 + exp(-25*d))**2, True))

Piecewise((0, d <= -0.286980112508514), (-15625*exp(-25*d)/(1 + exp(-25*d))**2 + 31250*exp(-50*d)/(1 + exp(-25*d))**3, True))

In [6]:
def objective_function(theta, X, y):
    n = len(X)
    predictions = X.dot(theta)
    errors = predictions - y
    cost = np.array([float(f(x)) for x in errors])
    return np.sum(cost) / n

def gradient(theta, X, y):
    m = len(theta)
    n = len(X)
    errors = X.dot(theta) - y
    gradi = np.array([float(f_grad(x)) for x in errors])
    return theta * (np.sum(gradi) / n)


In [7]:
# Initial guess for theta (weights)
initial_theta = np.zeros(X_train.shape[1])

display(objective_function(initial_theta, X_train, y_train))
display(gradient(initial_theta, X_train, y_train))
# Optimize using scipy.optimize.minimize
# model = minimize(fun=objective_function, x0=initial_theta, jac=gradient, args=(X_train, y_train), method='BFGS', options={'disp':True, 'return_all':True}),
model = minimize(fun=objective_function, x0=initial_theta, jac=gradient, args=(X_train, y_train), method='Nelder-Mead', options={'disp':True, 'return_all':True, 'maxiter':10000}),



20.851919349311835

array([0., 0., 0., 0., 0., 0., 0., 0.])

/Users/preetkaria/miniconda3/envs/mlClass/lib/python3.12/site-packages/scipy/optimize/_minimize.py:560: RuntimeWarning: Method Nelder-Mead does not use gradient information (jac).
  warn('Method %s does not use gradient information (jac).' % method,


Optimization terminated successfully.
         Current function value: 7.463586
         Iterations: 177
         Function evaluations: 315


In [8]:
display(model)
coef_df = pd.DataFrame({'Feature': train_cols, 'Coefficient': np.round(model[0].x[1:], 4)})
display(coef_df)
test_df['predicted_u_star'] = X_test.dot(model[0].x)
test_df['actual_cost'] = test_df.apply(lambda row: methods.cal_cost(row['c'], row['h'], row['u'], row['predicted_u_star']), axis=1)
print(f'Actual Mean cost: {test_df['actual_cost'].mean():.2f}, Actual Median cost: {test_df['actual_cost'].median():.2f}')
print(f'Optimal Mean cost: {test_df['optimal_cost'].mean():.2f}, Optimal Median cost: {test_df['optimal_cost'].median():.2f}')

(       message: Optimization terminated successfully.
        success: True
         status: 0
            fun: 7.463586457846256
              x: [ 3.973e-02  3.944e-01 -2.893e-01 -4.199e-01  3.165e-02
                  -1.705e-01 -2.735e-01  8.628e-01]
            nit: 177
           nfev: 315
  final_simplex: (array([[ 3.973e-02,  3.944e-01, ..., -2.735e-01,
                          8.628e-01],
                        [ 3.978e-02,  3.944e-01, ..., -2.735e-01,
                          8.628e-01],
                        ...,
                        [ 3.979e-02,  3.944e-01, ..., -2.735e-01,
                          8.628e-01],
                        [ 3.980e-02,  3.944e-01, ..., -2.735e-01,
                          8.628e-01]]), array([ 7.464e+00,  7.464e+00,  7.464e+00,  7.464e+00,
                         7.464e+00,  7.464e+00,  7.464e+00,  7.464e+00,
                         7.464e+00]))
        allvecs: [array([ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
                

,Feature,Coefficient
0,N,0.3944
1,n,-0.2893
2,mean_n,-0.4199
3,std_n,0.0316
4,alpha_hat,-0.1705
5,beta_hat,-0.2735
6,u_star_hat,0.8628


Actual Mean cost: 8.27, Actual Median cost: 5.99
Optimal Mean cost: 4.36, Optimal Median cost: 3.17


In [9]:
x = test_df['u'] - test_df['predicted_u_star']
display(len(x[x < 0]))
display(x[x < 0].mean())
test_df[['u', 'predicted_u_star']].head(50)

2744

-32.7199933385092

,u,predicted_u_star
0,373.970646,287.923615
1,161.389327,104.360252
2,143.176863,137.301166
3,108.552055,65.594933
4,82.809339,62.411517
5,87.637870,47.969557
6,106.970370,108.639240
7,334.042238,181.408139
8,233.670311,204.663038
9,53.291864,58.632082


Piecewise((-0.0066*d, d <= -0.28799426367831), (34/(1 + exp(-34*d)), True))

Piecewise((0.0066*(-d)**1.25, d <= 0), (0.0066*d**4, True))

TypeError: get_POI_cus_cost_expr_1() takes 2 positional arguments but 3 were given